In [32]:
import pandas as pd
import random
import datetime

def get_date():
    # Get today's date
    today = pd.Timestamp.today().normalize()

    # Get a date that is either yesterday or the day before yesterday, but not on a weekend
    for i in range(2, 7):
        date = today - pd.DateOffset(days=i)
        if date.dayofweek < 5:  # If the date is not on a weekend
            return date

def get_start_time(date):
    # Create a start time
    return datetime.datetime.combine(date, datetime.time(random.randint(20, 23), random.randint(0, 59))) if random.random() < 0.5 else datetime.datetime.combine(date + pd.DateOffset(days=1), datetime.time(random.randint(0, 1), random.randint(0, 59)))

def get_cutoff_time(date):
    # Create a cutoff time
    return datetime.datetime.combine(date + pd.DateOffset(days=1), datetime.time(random.randint(0, 5), 0))


def get_end_time(date):
    # Create an end time
    return datetime.datetime.combine(date + pd.DateOffset(days=1), datetime.time(random.randint(0, 7), random.randint(0, 59)))



In [33]:
import pandas as pd
import random
import datetime

def get_date():
    # Get today's date
    today = pd.Timestamp.today().normalize()

    # Get a date that is either yesterday or the day before yesterday, but not on a weekend
    for i in range(2, 7):
        date = today - pd.DateOffset(days=i)
        if date.dayofweek < 5:  # If the date is not on a weekend
            return date

def get_start_time(date):
    # Create a start time
    return datetime.datetime.combine(date, datetime.time(random.randint(20, 23), random.randint(0, 59))) if random.random() < 0.5 else datetime.datetime.combine(date + pd.DateOffset(days=1), datetime.time(random.randint(0, 1), random.randint(0, 59)))

def get_cutoff_time(date):
    # Create a cutoff time
    return datetime.datetime.combine(date + pd.DateOffset(days=1), datetime.time(random.randint(0, 5), 0))

def get_end_time(date):
    # Create an end time
    return datetime.datetime.combine(date + pd.DateOffset(days=1), datetime.time(random.randint(0, 7), random.randint(0, 59)))

In [34]:
def generate_data():
    data = []

    for _ in range(20):
        date = get_date()
        start_time = get_start_time(date)
        cutoff_time = get_cutoff_time(date)
        end_time = get_end_time(date)

        # Append the data to the list
        data.append([date, start_time, cutoff_time, end_time])

    # Create the DataFrame
    df = pd.DataFrame(data, columns=['Date', 'Start Time', 'Cutoff Time', 'End Time'])

    # Create the 'Buffer' column
    df['Buffer'] = (df['Cutoff Time'] - df['End Time']).apply(lambda x: x.total_seconds() / 3600)

    # Print the DataFrame
    print(df)


In [35]:

generate_data()

         Date          Start Time         Cutoff Time            End Time  \
0  2023-12-15 2023-12-15 23:34:00 2023-12-16 05:00:00 2023-12-16 01:20:00   
1  2023-12-15 2023-12-15 23:51:00 2023-12-16 00:00:00 2023-12-16 04:18:00   
2  2023-12-15 2023-12-15 23:10:00 2023-12-16 00:00:00 2023-12-16 02:11:00   
3  2023-12-15 2023-12-16 00:42:00 2023-12-16 02:00:00 2023-12-16 03:22:00   
4  2023-12-15 2023-12-16 00:06:00 2023-12-16 02:00:00 2023-12-16 01:34:00   
5  2023-12-15 2023-12-15 21:01:00 2023-12-16 00:00:00 2023-12-16 01:43:00   
6  2023-12-15 2023-12-15 21:31:00 2023-12-16 04:00:00 2023-12-16 04:28:00   
7  2023-12-15 2023-12-16 01:33:00 2023-12-16 02:00:00 2023-12-16 00:31:00   
8  2023-12-15 2023-12-16 01:23:00 2023-12-16 02:00:00 2023-12-16 05:22:00   
9  2023-12-15 2023-12-15 23:50:00 2023-12-16 04:00:00 2023-12-16 03:03:00   
10 2023-12-15 2023-12-15 22:59:00 2023-12-16 05:00:00 2023-12-16 04:08:00   
11 2023-12-15 2023-12-15 22:09:00 2023-12-16 04:00:00 2023-12-16 06:47:00   